In [16]:
%pip install elasticsearch pymysql

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [17]:
# docker run  --net esnet -p 9200:9200 -p 9300:9300 -e “discovery.type=single-node” elasticsearch:7.17.17

In [18]:
import pymysql
from elasticsearch import Elasticsearch

# MySQL connection parameters
mysql_host = '103.82.133.168'
mysql_port = 3306
mysql_user = 'root'
mysql_password = 'chiemdu123'
mysql_db = 'YuGiOh'

# Elasticsearch connection parameters
es_host = '103.82.27.204'
es_port = 9200
es_index = 'yugi_oh_index'

def fetch_mysql_data():
    # Establish MySQL connection
    connection = pymysql.connect(host=mysql_host,port=mysql_port,  user=mysql_user, password=mysql_password, db=mysql_db, charset='utf8mb4')
    cursor = connection.cursor()

    # Fetch data from MySQL
    query = "SELECT * FROM YuGiOh_cards"
    cursor.execute(query)
    result = cursor.fetchall()

    # Close MySQL connection
    cursor.close()
    connection.close()

    return result

def push_to_elasticsearch(data):
    # Establish Elasticsearch connection
    es = Elasticsearch([{'host': es_host, 'port': es_port,"scheme": "http"}])

    # Indexing data into Elasticsearch
    for record in data:
        document = {
            'Image_name': record[0],
            'Card_number': record[1],
            'Rarity': record[2],
            'Card_name': record[3],
            'Card_type': record[4],
            'Attribute': record[5],
            'Types': record[6],
            'Level': record[7],
            'Description': record[8],
        }

        # Index the document into Elasticsearch
        es.index(index=es_index, body=document)

In [19]:
# Fetch data from MySQL
mysql_data = fetch_mysql_data()

In [20]:
mysql_data.__len__()

8373

In [21]:
part_data = mysql_data[0:100]
part_data.__len__()

100

In [22]:
# Push data to Elasticsearch
push_to_elasticsearch(mysql_data)

/var/folders/cm/4m2_895j17v2wh5_3g0zqj9h0000gn/T/ipykernel_7438/2094240387.py:51: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.index(index=es_index, body=document)
